In [1]:
import gc
import time
import pandas as pd
import numpy as np
import csv
import pywt
import os
from sklearn.metrics import f1_score,hamming_loss
from sklearn.metrics import precision_recall_fscore_support as prf
import warnings
warnings.filterwarnings("ignore")
path = "/media/jdcloud/"

In [2]:
def preprocess_y(labels, y, num_class=10):
    bin_label = np.zeros((len(y), num_class)).astype('int8')
    for i in range(len(y)):
        label_nona = labels.loc[y[i]].dropna()
        for j in range(1, label_nona.shape[0]):
            bin_label[i, int(label_nona[j])] = 1
    return bin_label

In [17]:
def f(x):
    if x=="MALE":
        return 0
    elif x=="FEMALE":
        return 1
    else:
        return 2
labels_en = pd.read_csv(path + "kfold_labels_en.csv")
labels_en_drop = labels_en.drop_duplicates(["File_name"])
labels_en_drop.age.values.shape
labels_en_drop.sex.apply(lambda x:f(x)).values

array([0, 0, 0, ..., 1, 0, 0])

In [19]:
train = []

#train.append(pd.read_csv(path+"ensemble_csv/"+"densenet_attention_maxpooling_10net_fold0.csv").values)
#train.append(pd.read_csv(path+"ensemble_csv/"+"densenet_attention_maxpooling_10net_fold1.csv").values)
#train.append(pd.read_csv(path+"ensemble_csv/"+"densenet_attention_maxpooling_10net_fold2.csv").values)

path = "./"
# train.append(pd.read_csv(path+"ensemble_csv/"+"densenet_4block_10net_fold0.csv").values)
# train.append(pd.read_csv(path+"ensemble_csv/"+"densenet_4block_10net_fold1.csv").values)
train.append(pd.read_csv(path+"ensemble_csv/"+"densenet_4block_10net_fold2.csv").values) #1fold f0.822

#train.append(pd.read_csv(path+"ensemble_csv/"+"attention_one_net_fold0.csv").values)
#train.append(pd.read_csv(path+"ensemble_csv/"+"attention_one_net_fold1.csv").values)
train.append(pd.read_csv(path+"ensemble_csv/"+"attention_one_net_fold2.csv").values) # 1fold f0.813

# train.append(pd.read_csv(path+"ensemble_csv/"+"attention_1net_fold0.csv").values)
# train.append(pd.read_csv(path+"ensemble_csv/"+"attention_1net_fold1.csv").values)
# train.append(pd.read_csv(path+"ensemble_csv/"+"attention_1net_fold2.csv").values)

train.append(pd.read_csv(path+"ensemble_csv/"+"attention_10net_fold0.csv").values)
train.append(pd.read_csv(path+"ensemble_csv/"+"attention_10net_fold1.csv").values)
train.append(pd.read_csv(path+"ensemble_csv/"+"attention_10net_fold2.csv").values) # 3folds f0.817

train.append(pd.read_csv(path+"ensemble_csv/"+"densenet_f0819_10net_fold.csv").values) # 1fold f0.819

In [20]:
train.append(labels_en_drop.age.values)
train.append(labels_en_drop.sex.apply(lambda x:f(x)).values)

In [28]:
test = []

#test.append(pd.read_csv(path+"test_csv/"+"densenet_attention_maxpooling_10net_fold0.csv").values)
#test.append(pd.read_csv(path+"test_csv/"+"densenet_attention_maxpooling_10net_fold1.csv").values)
#test.append( pd.read_csv(path+"test_csv/"+"densenet_attention_maxpooling_10net_fold2.csv").values)

# test.append(pd.read_csv(path+"test_csv/"+"densenet_4block_10net_fold0.csv").values)
# test.append(pd.read_csv(path+"test_csv/"+"densenet_4block_10net_fold1.csv").values)
test.append(pd.read_csv(path+"test_csv/"+"densenet_4block_10net_fold2.csv").values)

#test.append(pd.read_csv(path+"test_csv/"+"attention_one_net_fold0.csv").values)
#test.append(pd.read_csv(path+"test_csv/"+"attention_one_net_fold1.csv").values)
test.append(pd.read_csv(path+"test_csv/"+"attention_one_net_fold2.csv").values)

# test.append(pd.read_csv(path+"test_csv/"+"attention_1net_fold0.csv").values)
# test.append(pd.read_csv(path+"test_csv/"+"attention_1net_fold1.csv").values)
# test.append(pd.read_csv(path+"test_csv/"+"attention_1net_fold2.csv").values)

test.append(pd.read_csv(path+"test_csv/"+"attention_10net_fold0.csv").values)
test.append(pd.read_csv(path+"test_csv/"+"attention_10net_fold1.csv").values)
test.append(pd.read_csv(path+"test_csv/"+"attention_10net_fold2.csv").values)

test.append(pd.read_csv(path+"test_csv/"+"densenet_f0819_10net_fold.csv").values)

In [24]:
import scipy.io as sio
def read_data_labels_info(data_path, split = "Train",preprocess=True):
    """ Read data """

    # Paths
    path_signals = os.path.join(data_path, split)

    # Read labels and one-hot encode
    labels = pd.read_csv(data_path+"REFERENCE.csv")
    #2156+224+672+654+180+826+534+504+1953
    
    # Read time-series data
    channel_files = os.listdir(path_signals)
    channel_files.sort()
    n_channels = 12#len(channel_files)

    # Initiate array
    list_of_channels = []
    i_ch = 0
    
    data_x = []
    data_y = []
    
    channel_name = ['V6', 'aVF', 'I', 'V4', 'V2', 'aVL', 'V1','II', 'aVR', 'V3', 'III', 'V5']
    channel_mid_name = ['II','aVR','V2','V5']
    channel_post_name = ['III','aVF','V3','V6']
      
    info = []
    
    for i_ch,fil_ch in enumerate(channel_files[:]):#tqdm

        labels_list = list(labels.iloc[i_ch].values[:])#.dropna()
        #print(labels_list)

        ecg = sio.loadmat(os.path.join(path_signals,fil_ch))
        a = 0
        for m in labels_list:
            if m in [0,1,2,3,4,5,6,7,8,9]:
                a += 1
        labels_list.append(a)
        labels_list.append(ecg['sex'][0])
        labels_list.append(ecg['age'][0][0])
        labels_list.append(ecg['I'].shape[1])
        
        
        info.append(labels_list)
    column_name = ['File_name','label1','label2','label3','label4','label5','label6',
                   'label7','label8','label9','label10','labels_num','sex','age','dataSize']
    info_pd =pd.DataFrame(columns=column_name,index=range(len(channel_files)),data=info)
    # Return 
    return info_pd

In [25]:
test_info = read_data_labels_info(data_path="/media/jdcloud/",split="Val")

In [29]:
test.append(test_info.age.values)
test.append(test_info.sex.apply(lambda x:f(x)).values)

In [337]:
import lightgbm as lgb
# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',#'multiclass',
    'metric': 'binary_logloss',
    'num_class': 1,
    'num_leaves': 100,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0,
    'seed':2019
}

In [57]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression, ElasticNet, Lasso, RidgeCV
from sklearn.ensemble import GradientBoostingClassifier
from mlknn import MLkNN

gbm = GradientBoostingClassifier(learning_rate=0.005,n_estimators=100,max_depth=5,min_samples_leaf=20,
                                min_samples_split=600,subsample=0.7,random_state=2019)
LR = LogisticRegression(penalty="l2",C=1.0)
Eln = ElasticNet()
Las = Lasso(alpha=0.2)
LRR = RidgeCV()

pre_type = "db6"#"db6"#"sym"
labels = pd.read_csv("/media/jdcloud/" + "REFERENCE.csv")

index = np.arange(6689)
y_train = preprocess_y(labels, index)

x_train = np.hstack(train[:-2])
x_train = np.column_stack([x_train,train[-1]])
x_train = np.column_stack([x_train,train[-2]])
#train_x, valid_x, train_y, valid_y = train_test_split(x, y, test_size=0.333, random_state=0)
#train = lgb.Dataset(x_train,label=y_train)
#valid = lgb.Dataset(valid_x, label=valid_y
#gbm = lgb.train(params,train,num_boost_round=1000,#valid_sets=valid,early_stopping_rounds=5)

#clf = MLkNN(k=8)

clf = OneVsRestClassifier(LR)
clf.fit(x_train,y_train)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
          n_jobs=None)

In [58]:
# LR_clf = joblib.load("LR_ensemble.pkl")
# MLkNN_clf = joblib.load("MLkNN_ensemble.pkl")

# y_pred_LR = LR_clf.predict(x_train)
# y_pred_proba_LR = LR_clf.predict_proba(x_train)

# y_pred_MLkNN = MLkNN_clf.predict(x_train).toarray()
# y_pred_proba_MLkNN = MLkNN_clf.predict_proba(x_train).toarray()

# #y_pred_MLkNN[:,7] = y_pred_LR[:,7]
# #y_pred_proba_MLkNN[:,7] = y_pred_proba_LR[:,7]
# y_pred = y_pred_MLkNN
# y_pred_proba_train = y_pred_proba_MLkNN

y_pred = clf.predict(x_train)#.toarray()
y_pred_proba_train = clf.predict(x_train)#.toarray()
print(" train data f1_score  :", f1_score(y_train, y_pred, average='macro'))
for i in range(10):
    print("f1 score of ab {} is {}".format(i, f1_score(y_train[:, i], y_pred[:, i], average='macro')))
    
print(" train data hamming_loss  :", hamming_loss(y_train, y_pred)) 
print(" train data precision recall f1  :", prf(y_train, y_pred,average="samples"))# 'micro', 'weighted'

 train data f1_score  : 0.9425787389674495
f1 score of ab 0 is 0.9676716335176512
f1 score of ab 1 is 0.9870324128569159
f1 score of ab 2 is 0.9664490036021178
f1 score of ab 3 is 0.985171184676515
f1 score of ab 4 is 0.9591266834502481
f1 score of ab 5 is 0.9724710614168124
f1 score of ab 6 is 0.9651989734821169
f1 score of ab 7 is 0.9573247757461305
f1 score of ab 8 is 0.9528357405368497
f1 score of ab 9 is 0.9599398212397567
 train data hamming_loss  : 0.012498131260278069
 train data precision recall f1  : (0.9458314645936113, 0.944941944485972, 0.9417092566741244, None)


In [153]:
out = np.hstack(test[:-2])
out = np.column_stack([out,test[-1]])
out = np.column_stack([out,test[-2]])
# LR_clf = joblib.load("LR_ensemble.pkl")
# MLkNN_clf = joblib.load("MLkNN_ensemble.pkl")

# y_pred_LR = LR_clf.predict(out)
# y_pred_proba_LR = LR_clf.predict_proba(out)

# y_pred_MLkNN = MLkNN_clf.predict(out).toarray()
# y_pred_proba_MLkNN = MLkNN_clf.predict_proba(out).toarray()

# y_pred_MLkNN[:,7] = y_pred_LR[:,7]
# y_pred_proba_MLkNN[:,7] = y_pred_proba_LR[:,7]
# y_pred = y_pred_MLkNN
# y_pred_proba = y_pred_proba_MLkNN

y_pred = clf.predict(out)#.toarray()
y_pred_proba = clf.predict_proba(out)#.toarray()

classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

pred = []
for j in range(y_pred.shape[0]):
    pred.append([classes[i] for i in range(10) if y_pred[j][i] == 1])

''' ''' 
for i, val in enumerate(pred):
    if val == []:
        pass
        #for i_p, val_p in enumerate(y_pred_proba[i]):
        #    if val_p >= 0.4:
        #        pred[i].append(i_p)    # f1 == 0.832
                
        if y_pred_proba[i][np.argmax(y_pred_proba[i])] >= 0.3:
            pred[i] = [np.argmax(y_pred_proba[i])]     # f1 == 0.833  0.4

val_dataset_path = '/media/jdcloud' + "/Val/"
val_files = os.listdir(val_dataset_path)
val_files.sort()

with open('jupyter_answers_densenet_{}_0809.csv'.format(pre_type), 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['File_name', 'label1', 'label2',
                     'label3', 'label4', 'label5', 'label6', 'label7', 'label8', 'label9', 'label10'])
    count = 0
    for file_name in val_files:
        if file_name.endswith('.mat'):

            record_name = file_name.strip('.mat')
            answer = []
            answer.append(record_name)

            result = pred[count]

            answer.extend(result)
            for i in range(10 - len(result)):
                answer.append('')
                
            count += 1
            writer.writerow(answer)
    csvfile.close()

In [69]:
from sklearn.externals import joblib
# Save
# joblib.dump(clf,"MLkNN8_ensemble.pkl")#LR_ensemble

In [466]:
thr = 0.6
#out1 = thr * (0.1 * train[0] + 0.3 * train[1] + 0.6 * train[2])
#out2 = (1 - thr) * (1.0 * train[3])
thr0 = 0; thr1 = 0.4; thr2 = 0.; thr3 = 0.0; thr4 = 0.; thr5 = 0.6;
train_y = thr0*train[0] + thr1*train[1] + thr2*train[2] + thr3*train[3] + thr4*train[4] + thr5*train[5]

threshold = np.arange(0.1, 0.9, 0.1)
acc = []
accuracies = []
best_threshold = np.zeros(train_y.shape[1])

for i in range(train_y.shape[1]):
    y_prob = np.array(train_y[:, i])
    for j in threshold:
        y_pred = [1 if prob >= j else 0 for prob in y_prob]
        acc.append(f1_score(y_train[:, i], y_pred, average='macro'))
    acc = np.array(acc)
    index = np.where(acc == acc.max())
    accuracies.append(acc.max())
    best_threshold[i] = threshold[index[0][0]]
    acc = []

print("best_threshold :", best_threshold)

y_pred = np.array([[1 if train_y[i, j] >= best_threshold[j] else 0 for j in range(train_y.shape[1])]
          for i in range(len(train_y))])
print(" train data f1_score  :", f1_score(y_train, y_pred, average='macro'))

for i in range(10):
    print("f1 score of ab {} is {}".format(i, f1_score(y_train[:, i], y_pred[:, i], average='macro')))
print(" train data hamming_loss  :", hamming_loss(y_train, y_pred)) 
print(" train data precision recall f1  :", prf(y_train, y_pred,average="samples"))# 'micro', 'weighted'

best_threshold : [0.7 0.5 0.5 0.4 0.6 0.2 0.3 0.4 0.5 0.3]
 train data f1_score  : 0.9226479088215832
f1 score of ab 0 is 0.9576243119144202
f1 score of ab 1 is 0.9794953051205606
f1 score of ab 2 is 0.951682745472497
f1 score of ab 3 is 0.982355844958436
f1 score of ab 4 is 0.9477985152051004
f1 score of ab 5 is 0.9721827424838709
f1 score of ab 6 is 0.958814994471973
f1 score of ab 7 is 0.9415063228088438
f1 score of ab 8 is 0.9398197061954244
f1 score of ab 9 is 0.9297211830048123
 train data hamming_loss  : 0.016474809388548364
 train data precision recall f1  : (0.9262096975133303, 0.9349429411471569, 0.9257665392364153, None)


In [137]:
best_threshold

array([0.6, 0.6, 0.5, 0.5, 0.6, 0.2, 0.2, 0.7, 0.5, 0.4])

In [395]:
out = thr0 * test[0]# + thr1 * test[1] + thr2 * test[2]# + thr3 * test[3] + thr4 * test[4] + thr5 * test[5]

y_pred_test = np.array(
    [[1 if out[i, j] >= best_threshold[j] else 0 for j in range(out.shape[1])] for i in range(len(out))])

classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

test_y = y_pred_test

y_pred = [[1 if test_y[i, j] >= best_threshold[j] else 0 for j in range(test_y.shape[1])]
          for i in range(len(test_y))]
pred = []
for j in range(test_y.shape[0]):
    pred.append([classes[i] for i in range(10) if y_pred[j][i] == 1])
    
for i, val in enumerate(pred):
    ''' 
    if 0 in val and len(val) > 1:
        flag = 0
        for j in val:
            if (test_y[i][0] - best_threshold[0]) > (test_y[i][j] - best_threshold[j]):
                pass
            else:
                flag = 1
        if flag == 1:
            pred[i] = val[1:]
        else:
            pred[i] = val[0]
    '''
    if len(val) == 0:
        pred[i] = [np.argmin(np.abs(best_threshold - out_test[i]))]
        
val_dataset_path = path + "/Val/"
val_files = os.listdir(val_dataset_path)
val_files.sort()

with open('jupyter_answers_densenet_{}_0803.csv'.format(pre_type), 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['File_name', 'label1', 'label2',
                     'label3', 'label4', 'label5', 'label6', 'label7', 'label8', 'label9', 'label10'])
    count = 0
    for file_name in val_files:
        if file_name.endswith('.mat'):

            record_name = file_name.strip('.mat')
            answer = []
            answer.append(record_name)

            result = pred[count]

            answer.extend(result)
            for i in range(10 - len(result)):
                answer.append('')
            count += 1
            writer.writerow(answer)
    csvfile.close()

# 0.833 and 0.842 ensemble 

In [334]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(penalty="l2",C=1.0)


pre_type = "db6"#"db6"#"sym"
labels = pd.read_csv(path + "REFERENCE.csv")

index = np.arange(6689)
y_train = preprocess_y(labels, index)

x_train = np.hstack([train_y,y_pred_proba_train])

clf = OneVsRestClassifier(LR)
clf.fit(x_train,y_train)

y_pred = clf.predict(x_train)

print(" train data f1_score  :", f1_score(y_train, y_pred, average='macro'))
for i in range(10):
    print("f1 score of ab {} is {}".format(i, f1_score(y_train[:, i], y_pred[:, i], average='macro')))

 train data f1_score  : 0.9211840609081451
f1 score of ab 0 is 0.9603084096823336
f1 score of ab 1 is 0.9793386425479889
f1 score of ab 2 is 0.9513702179183157
f1 score of ab 3 is 0.9822910749256628
f1 score of ab 4 is 0.9444799508628059
f1 score of ab 5 is 0.9696737288321967
f1 score of ab 6 is 0.9584407222873685
f1 score of ab 7 is 0.9350094053104944
f1 score of ab 8 is 0.9451176555412449
f1 score of ab 9 is 0.930177490596017


In [151]:
out = 0.4 * test[0] + 0.6 * test[1]
x_test = np.hstack([out,y_pred_proba])
y_pred = clf.predict(x_test)
y_pred_proba = clf.predict_proba(x_test)

classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

pred = []
for j in range(y_pred.shape[0]):
    pred.append([classes[i] for i in range(10) if y_pred[j][i] == 1])

''' ''' 
for i, val in enumerate(pred):
    if val == []:
        pass
        #for i_p, val_p in enumerate(y_pred_proba[i]):
        #    if val_p >= 0.4:
        #        pred[i].append(i_p)    # f1 == 0.832
                
        if y_pred_proba[i][np.argmax(y_pred_proba[i])] >= 0.4:
            pred[i] = [np.argmax(y_pred_proba[i])]     # f1 == 0.833


val_dataset_path = path + "/Val/"
val_files = os.listdir(val_dataset_path)
val_files.sort()

with open('jupyter_answers_densenet_{}_0806.csv'.format(pre_type), 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['File_name', 'label1', 'label2',
                     'label3', 'label4', 'label5', 'label6', 'label7', 'label8', 'label9', 'label10'])
    count = 0
    for file_name in val_files:
        if file_name.endswith('.mat'):

            record_name = file_name.strip('.mat')
            answer = []
            answer.append(record_name)

            result = pred[count]

            answer.extend(result)
            for i in range(10 - len(result)):
                answer.append('')
            count += 1
            writer.writerow(answer)
    csvfile.close()

ValueError: X has 20 features per sample; expecting 62

# quarter-final data test

In [64]:
quarter_labels= pd.read_csv("/media/uuser/data/final_codes/final_run_semi/reference.csv")
quarter_index = np.arange(6500)
quarter_y_train = preprocess_y(quarter_labels, quarter_index)

In [ ]:
quarter_train = []
# quarter_train.append(pd.read_csv(path+"quarter_final/"+"attention_one_net_fold2.csv").values) 
# quarter_train.append(pd.read_csv(path+"quarter_final/"+"densenet_f0819_10net_fold.csv").values) 
# quarter_train.append(pd.read_csv(path+"quarter_final/"+"densenet_attention_10net_fold2.csv").values) 

# quarter_train.append(pd.read_csv(path+"quarter_final/"+"densenet_4block_10net_fold0.csv").values)
# quarter_train.append(pd.read_csv(path+"quarter_final/"+"densenet_4block_10net_fold1.csv").values)
quarter_train.append(pd.read_csv(path+"quarter_final/"+"densenet_4block_10net_fold2.csv").values)

#quarter_train.append(pd.read_csv(path+"quarter_final/"+"attention_one_net_fold0.csv").values)
#quarter_train.append(pd.read_csv(path+"quarter_final/"+"attention_one_net_fold1.csv").values)
quarter_train.append(pd.read_csv(path+"quarter_final/"+"attention_one_net_fold2.csv").values)

quarter_train.append(pd.read_csv(path+"quarter_final/"+"attention_10net_fold0.csv").values)
quarter_train.append(pd.read_csv(path+"quarter_final/"+"attention_10net_fold1.csv").values)
quarter_train.append(pd.read_csv(path+"quarter_final/"+"attention_10net_fold2.csv").values)

quarter_train.append(pd.read_csv(path+"quarter_final/"+"densenet_f0819_10net_fold.csv").values)

''' '''
# quarter_train_y = thr0*quarter_train[0] + thr1*quarter_train[1] + thr2*quarter_train[2] + \
#                 thr3*quarter_train[3] + thr4*quarter_train[4] + thr5*quarter_train[5]

# print("best_threshold :", best_threshold)

# quarter_y_pred = np.array([[1 if quarter_train_y[i, j] >= best_threshold[j] else 0 
#                             for j in range(quarter_train_y.shape[1])]
#           for i in range(len(quarter_train_y))])

# Restore
# LR_clf = joblib.load("LR_ensemble.pkl")
# MLkNN_clf = joblib.load("MLkNN_ensemble.pkl")

# quarter_x_train = np.hstack(quarter_train)
# quarter_y_pred_LR = LR_clf.predict(quarter_x_train)
# quarter_y_pred_MLkNN = MLkNN_clf.predict(quarter_x_train).toarray()

# quarter_y_pred_MLkNN[:,7] = quarter_y_pred_LR[:,7]
# quarter_y_pred = quarter_y_pred_MLkNN

quarter_y_pred = clf.predict(quarter_x_train).toarray()
print(" train data f1_score  :", f1_score(quarter_y_train, quarter_y_pred, average='macro'))

for i in range(10):
    print("f1 score of ab {} is {}".format(i, f1_score(quarter_y_train[:, i], quarter_y_pred[:, i], average='macro')))
    
print(" train data hamming_loss  :", hamming_loss(quarter_y_train, quarter_y_pred)) 
print(" train data precision recall f1  :", prf(quarter_y_train, quarter_y_pred,average="samples"))# 'micro', 'weighted'

In [496]:
Counter(quarter_y_train[:,7])

Counter({0: 6276, 1: 224})

In [76]:
from collections import Counter

Counter(quarter_y_pred[:,7])

Counter({0: 6272, 1: 228})

In [524]:
(0.843 * 10 - 0.498 + 0.8) / 10

0.8732

In [ ]:
 train data f1_score  : 0.9226479088215832
f1 score of ab 0 is 0.9576243119144202
f1 score of ab 1 is 0.9794953051205606
f1 score of ab 2 is 0.951682745472497
f1 score of ab 3 is 0.982355844958436
f1 score of ab 4 is 0.9477985152051004
f1 score of ab 5 is 0.9721827424838709
f1 score of ab 6 is 0.958814994471973
f1 score of ab 7 is 0.9415063228088438
f1 score of ab 8 is 0.9398197061954244
f1 score of ab 9 is 0.9297211830048123